In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
# Ielādē apmācību un testa datus
df = pd.read_csv('dati_fin.csv', sep=';')
df_test = pd.read_csv('2024_fin.csv', sep=';')

In [ ]:
# No apmācības un testa datu kopām atdala kolonnu 'Darījuma summa, EUR'
x_train = df.drop(columns=['Darījuma summa, EUR'])
y_train = df['Darījuma summa, EUR']
x_test = df_test.drop(columns=['Darījuma summa, EUR']) 
y_test_actual = df_test['Darījuma summa, EUR']

In [ ]:
# Definē hiperparametru režģi GridSearchCV funkcijai
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 15, 20],
    'max_features': ['sqrt', 'log2', None]
}

# Izveido gadījuma meža regresijas modeli
rf_reg = RandomForestRegressor(random_state=42)

# Ar GridSearchCV veic labāko parametru meklēšanu
grid_search = GridSearchCV(rf_reg, param_grid, scoring='neg_root_mean_squared_error')
grid_search.fit(x_train, y_train) # Apmāca modeli ar apmācības datiem

# Iegūst un izdrukā labākos parametrus
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 200}


In [ ]:
# Izveido gadījuma meža regresijas modeli, izmantojot labākos atrastos parametrus
best_rf_reg = RandomForestRegressor(**best_params, random_state=42)
# Apmāca šo modeli ar apmācību datiem
best_rf_reg.fit(x_train, y_train)

RandomForestRegressor(max_depth=15, max_features='sqrt', n_estimators=200,
                      random_state=42)

In [14]:
# Izmanto apmācīto modeli, lai prognozētu cenas testa datiem
y_test_pred = best_rf_reg.predict(x_test)

In [15]:
# Novērtē modeļa precizitāti uz testa datiem
mae = mean_absolute_error(y_test_actual, y_test_pred)
mse = mean_squared_error(y_test_actual, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_actual, y_test_pred)

# Aprēķina relatīvās kļūdas
relative_errors = np.abs(y_test_pred - y_test_actual) / y_test_actual
mre = np.max(relative_errors)
are = np.mean(relative_errors)

In [16]:
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")
print(f"MRE: {mre}")
print(f"ARE: {are}")

MAE: 10466.177906204894
MSE: 233058338.0116693
RMSE: 15266.248327983838
R^2: 0.7483639646121913
MRE: 8.658668774163239
ARE: 0.3489430581666273
